In [49]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.naive_bayes import MultinomialNB


In [9]:
df=pd.read_csv('./res/df_inne_choosen')
X=df.drop('Moda',axis=1)
y=df['Moda']
scaler=MinMaxScaler()
X_scaled=scaler.fit_transform(X)
X=pd.DataFrame(X_scaled,columns=X.columns)


In [61]:
print(X.columns)

Index(['Wzrost', 'gry_czas', 'Zajecie__sluchanie muzyki', 'Wakacje__Praca',
       'Rod_prz_sam__Przyjaciele', 'Napoj__soki', 'Napoj__woda'],
      dtype='object')


In [ ]:
comb=list(combinations(X.columns,2))
pvals={}
for col1,col2 in comb:
    crosstab=pd.crosstab(X[col1],X[col2])
    chi2,p,_,_=chi2_contingency(crosstab)
    pvals[(col1,col2)]=p

In [5]:
for key,value in pvals.items():
    if value<0.05: print(key,value)

('Napoj__soki', 'Napoj__woda') 0.027766816664195774


In [6]:
contingency_table=pd.crosstab(X['Napoj__soki'],X['Napoj__woda'])
chi2,p,_,_=chi2_contingency(contingency_table)
n = contingency_table.sum().sum()
phi2 = chi2 / n 
rcorr = min(contingency_table.shape) - 1
cramers_v = (phi2 / rcorr) ** 0.5
print(cramers_v)

0.40863470634825333


In [62]:
X.drop('Napoj__soki',axis=1,inplace=True)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

In [64]:
param_dict={
    'alpha':[float(x) for x in [*list(np.linspace(0,1,50)),*list(np.linspace(1,10,50)),1]],
    'fit_prior':[True, False],
    'class_prior':[None],
    'force_alpha':[True,False]
}

In [65]:
searcher=GridSearchCV(estimator=MultinomialNB(),param_grid=param_dict, cv=5, scoring='f1_weighted', n_jobs=-1)
searcher.fit(X, y)
print('parametr: ',searcher.best_params_)
print('najlepszy f1: ',searcher.best_score_)

/home/mysza/pythonProject/.venv/lib/python3.10/site-packages/sklearn/naive_bayes.py:897: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/home/mysza/pythonProject/.venv/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/mysza/pythonProject/.venv/lib/python3.10/site-packages/sklearn/naive_bayes.py:897: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/home/mysza/pythonProject/.venv/lib/python3.10/site-packages/sklearn/naive_bayes.py:897: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/home/mysza/pythonProject/.venv/lib/python3.10/site-packages/sklearn/naive_bayes.py:897: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/home/mysza/pythonProject/.venv/lib/python3.10/site-package

parametr:  {'alpha': 0.3469387755102041, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}
najlepszy f1:  0.715111111111111


In [67]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(f1_score(y_test, y_pred, average='weighted'))

0.4202020202020202


In [48]:
print(param_dict['var_smoothing'])

[1e-50, 0.010101010101010102, 0.020202020202020204, 0.030303030303030304, 0.04040404040404041, 0.05050505050505051, 0.06060606060606061, 0.07070707070707072, 0.08080808080808081, 0.09090909090909091, 0.10101010101010102, 0.11111111111111112, 0.12121212121212122, 0.13131313131313133, 0.14141414141414144, 0.15151515151515152, 0.16161616161616163, 0.17171717171717174, 0.18181818181818182, 0.19191919191919193, 0.20202020202020204, 0.21212121212121213, 0.22222222222222224, 0.23232323232323235, 0.24242424242424243, 0.25252525252525254, 0.26262626262626265, 0.27272727272727276, 0.2828282828282829, 0.29292929292929293, 0.30303030303030304, 0.31313131313131315, 0.32323232323232326, 0.33333333333333337, 0.3434343434343435, 0.3535353535353536, 0.36363636363636365, 0.37373737373737376, 0.38383838383838387, 0.393939393939394, 0.4040404040404041, 0.4141414141414142, 0.42424242424242425, 0.43434343434343436, 0.4444444444444445, 0.4545454545454546, 0.4646464646464647, 0.4747474747474748, 0.48484848484

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=97)

mnb=MultinomialNB(**searcher.best_params_)
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
print(f1_score(y_test, y_pred, average='weighted'))

0.8333333333333334


In [86]:
from joblib import dump

dump(mnb, './res/MultinomialNB.joblib')

['./res/MultinomialNB.joblib']

In [88]:
final_mnb=MultinomialNB(**searcher.best_params_)
final_mnb.fit(X, y)
dump(final_mnb,'./res/final_mnb.joblib')

['./res/final_mnb.joblib']